In [1]:
import os
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tabulate import tabulate

**Load The Data**

In [2]:
current_dir = os.getcwd() 
relative_path_train = os.path.join('..', 'data', 'preprocessed_train_data.csv')
relative_path_test = os.path.join('..', 'data', 'preprocessed_test_data.csv')

preprocessed_train_data = pd.read_csv(os.path.join(current_dir, relative_path_train))
preprocessed_test_data = pd.read_csv(os.path.join(current_dir, relative_path_test))

**Data Spliting**

In [3]:
x_train = preprocessed_train_data.drop(["satisfaction"], axis = 1)
y_train = preprocessed_train_data["satisfaction"]

x_test = preprocessed_test_data.drop(["satisfaction"], axis = 1)
y_test = preprocessed_test_data["satisfaction"]

#y_test = y_test.replace({"satisfied":1, "neutral or dissatisfied":0})

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

**Create a list of classifiers**

In [4]:
classifiers = [
    ("Logistic Regression", LogisticRegression()),
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("Gradient Boosting", GradientBoostingClassifier()),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Gaussian Naive Bayes", GaussianNB()),
    ("Multi-layer Perceptron", MLPClassifier()),
    ("XGBoost", XGBClassifier()),
    ("CatBoost", CatBoostClassifier()), 
    ("AdaBoost", AdaBoostClassifier())
]

In [5]:
# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=["Classifier", "Balanced Accuracy", "Training Accuracy", "Testing Accuracy", "F1 Score", "Precision", "Recall"])

**Train and evaluate each classifier**

In [6]:
for name, clf in classifiers:
    print(name)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    y_prob = clf.predict_proba(x_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_prob)
    accuracy_train = accuracy_score(y_train, clf.predict(x_train))
    accuracy_test = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Create a temporary Series with results for this classifier
    new_row = pd.Series({
        "Classifier": name,
        "Balanced Accuracy": auc_score,
        "Training Accuracy": accuracy_train,
        "Testing Accuracy": accuracy_test,
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall
    })

    # Append the Series as a new row to the results DataFrame
    results_df = pd.concat([results_df, new_row.to_frame().T], ignore_index=True)

# Sort the DataFrame by Balanced Accuracy in descending order
results_df = results_df.sort_values(by="Balanced Accuracy", ascending=False)

# Print the results table with styling
styled_results = results_df.style.background_gradient(cmap='Blues', subset=["Balanced Accuracy", "Training Accuracy", "Testing Accuracy", "F1 Score", "Precision", "Recall"])
#styled_results = styled_results.hide_index() 

Logistic Regression
Decision Tree
Random Forest
Gradient Boosting
K-Nearest Neighbors
Gaussian Naive Bayes
Multi-layer Perceptron


/home/donia/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


XGBoost
CatBoost
Learning rate set to 0.074823
0:	learn: 0.5925347	total: 317ms	remaining: 5m 16s
1:	learn: 0.4853586	total: 431ms	remaining: 3m 35s
2:	learn: 0.4290905	total: 613ms	remaining: 3m 23s
3:	learn: 0.3833252	total: 737ms	remaining: 3m 3s
4:	learn: 0.3513727	total: 821ms	remaining: 2m 43s
5:	learn: 0.3089257	total: 964ms	remaining: 2m 39s
6:	learn: 0.2883640	total: 1.06s	remaining: 2m 29s
7:	learn: 0.2711136	total: 1.21s	remaining: 2m 30s
8:	learn: 0.2577476	total: 1.31s	remaining: 2m 24s
9:	learn: 0.2402199	total: 1.37s	remaining: 2m 15s
10:	learn: 0.2314451	total: 1.46s	remaining: 2m 11s
11:	learn: 0.2185457	total: 1.63s	remaining: 2m 14s
12:	learn: 0.2103959	total: 1.75s	remaining: 2m 13s
13:	learn: 0.2041481	total: 1.94s	remaining: 2m 16s
14:	learn: 0.1956322	total: 2.05s	remaining: 2m 14s
15:	learn: 0.1903867	total: 2.16s	remaining: 2m 12s
16:	learn: 0.1835245	total: 2.31s	remaining: 2m 13s
17:	learn: 0.1753302	total: 2.45s	remaining: 2m 13s
18:	learn: 0.1715682	total: 

In [7]:
styled_results

,Classifier,Balanced Accuracy,Training Accuracy,Testing Accuracy,F1 Score,Precision,Recall
8,CatBoost,0.994832,0.973658,0.962786,0.956157,0.972635,0.940229
6,Multi-layer Perceptron,0.993478,0.967460,0.958993,0.951721,0.967470,0.936476
2,Random Forest,0.993258,0.999962,0.961127,0.954160,0.971540,0.937391
3,Gradient Boosting,0.987055,0.940878,0.941216,0.930771,0.946447,0.915606
9,AdaBoost,0.975945,0.925566,0.924347,0.911297,0.922449,0.900412
7,XGBoost,0.973716,0.972744,0.867025,0.821432,0.976909,0.708650
4,K-Nearest Neighbors,0.972847,0.953139,0.932367,0.919161,0.949283,0.890892
1,Decision Tree,0.942258,1.000000,0.943349,0.934365,0.934450,0.934279
0,Logistic Regression,0.923548,0.874192,0.871568,0.847978,0.866826,0.829931
5,Gaussian Naive Bayes,0.913202,0.849515,0.845060,0.817768,0.830424,0.805492


In [10]:
'''
CatBoost achieved the highest accuracy, F1 Score & Balanced Accuracy
'''

'\nCatBoost achieved the highest accuracy, F1 Score & Balanced Accuracy\n'